In [18]:
with open('./test.txt', 'r', encoding='utf-8') as f:
    one_txt = f.readlines()

# one_txt = sorted(one_txt, key=lambda x: float(x.split(' ')[0]))

with open('./test_v4/test/labels/MP_KSC_000003.txt', 'r', encoding='utf-8') as f:
    two_txt = f.readlines()
# two_txt = sorted(two_txt, key=lambda x: float(x.split(' ')[0]))
print(one_txt)
print(two_txt)

['5 0.107461 0.706085 0.204922 0.204057\n', '3 0.371609 0.825009 0.0743971 0.123901\n', '0 0.427561 0.776905 0.0207535 0.106983\n']
['2 0.13206 0.61104 0.26411 0.26089\n', '3 0.47965 0.64759 0.07841 0.08259\n', '2 0.95314 0.62029 0.05014 0.0554\n', '2 0.98747 0.63157 0.02505 0.06889\n']


In [16]:
with open('./test.txt', 'r', encoding='utf-8') as f:
    one_txt = [[float(num) for num in line.strip().split()] for line in f.readlines()]
    one_txt = list(map(lambda line: [round(num, 3) for num in line], one_txt))
    one_txt = sorted(one_txt, key=lambda x: (x[0], x[1], x[2], x[3], x[4]))

with open('./test_v4/test/labels/MP_KSC_000041.txt', 'r', encoding='utf-8') as f:
    two_txt = [[num for num in line.strip().split()] for line in f.readlines()]
    two_txt = list(map(lambda line: [num for num in line], two_txt))
    two_txt = sorted(two_txt, key=lambda x: (x[0], x[1], x[2], x[3], x[4]))

two_txt

[['0', '0.42837', '0.77691', '0.01719', '0.10833'],
 ['3', '0.3714', '0.82447', '0.07405', '0.12765'],
 ['5', '0.13839', '0.79229', '0.27677', '0.22227']]

In [58]:
import math, os, shutil, sys

# check_txt 이 함수를 만들게 된 배경은 predict 한 결과 txt파일이 없는 경우도 많아서 그땐 사람 검수 필요.

# os.listdir 해서 나온 리스트 값만 필요
# correct_ls = 정답 txt리스트
# predict_ls = predict 한 txt 리스트
# copy_path = 이미지 원본있는 폴더dir <- 버전이 계속 바뀌다보니.. 지정필요
# ex) ./datas_v4/test/images/
def check_txt(correct_ls, predict_ls, org_img_path):
    check = []
    
    # copy_path 경로가 './datas_v4/test/images/' 이런 형식이 아닐때. '/'맨마지막에 붙인다.
    if not org_img_path.endswith('/'):
        org_img_path = org_img_path + '/'

    # prdict txt파일 없는거 파악
    for _ in correct_ls:
        
        if _ not in predict_ls:
            check.append(_)

    # txt파일이 동일하므로 종료
    check = []
    if len(check) == 0:
        sys.exit('txt파일개수 동일합니다.')
    
    for _ in check:
        img_copy = org_img_path + _.replace('.txt', '.jpg')
        shutil.copy(img_copy, './human_check/img/')


# prdict txt파일이랑 test txt파일 비교해서 동일하지 않으면 human_check 폴더로 이동.
# 이미지는 원본이 아니라 predict 한 결과 이미지 이동.
# ex) one_path = './runs/detect/predict3/labels/MP_KSC_000041.txt'
#     two_path = './test_v4/test/labels/MP_KSC_000041.txt'
def model_verify(predict_path, correct_path, error_rate=5):

    file_name = predict_path.split('/')[-1]
    predict_img = '/'.join(predict_path.split('/')[:-2]) + '/' + file_name.replace('txt', 'jpg')

    # 첫번째 프레임
    with open(predict_path, 'r', encoding='utf-8') as f:
        one_txt = f.readlines()
    one_txt = sorted(one_txt, key=lambda x: float(x.split(' ')[0]))

    # 두번째 프레임
    with open(correct_path, 'r', encoding='utf-8') as f:
        two_txt = f.readlines()
    two_txt = sorted(two_txt, key=lambda x: float(x.split(' ')[0]))

    if len(one_txt) == len(two_txt):
        for one, two in zip(one_txt, two_txt):
            false_count = 0
         
            # txt 파일 정보
            one_cl = one.split(' ')[0]
            one_x = float(one.split(' ')[1])
            one_y = float(one.split(' ')[2])
            one_w = float(one.split(' ')[3])
            one_h = float(one.split(' ')[4])

            two_cl = two.split(' ')[0]
            two_x = float(two.split(' ')[1])
            two_y = float(two.split(' ')[2])
            two_w = float(two.split(' ')[3])
            two_h = float(two.split(' ')[4])

            # 클래스명이 같을때
            if one_cl == two_cl:

                # 오차율 공식
                # x_acc = int(abs(two_x - one_x) / one_x * 100)
                # y_acc = int(abs(two_y - one_y) / one_y * 100)
                # w_acc = int(abs(two_w - one_w) / one_w * 100)
                # h_acc = int(abs(two_h - one_h) / one_h * 100)

                x_acc = int(abs(round(one_x, 3) - round(two_x, 3))) * 100
                y_acc = int(abs(round(one_y, 3) - round(two_y, 3))) *100
                w_acc = int(abs(round(one_w, 3) - round(two_w, 3))) * 100
                h_acc = int(abs(round(one_h, 3) - round(two_h, 3))) * 100

                if x_acc <= error_rate and y_acc <= error_rate and w_acc <= error_rate and h_acc <= error_rate: 

                    # print('동일합니다.')
                    # print(x_acc, y_acc, w_acc, h_acc)
                    pass
                               
                else:
                    # print('다릅니다.')
                    # print(x_acc, y_acc, w_acc, h_acc)
                    false_count += 1              

        # txt 파일 한 행씩 비교 후 same_count가 0이면 동일한 이미지가 아니므로 카피.
        if false_count > 0:   
            shutil.copy(predict_img, './human_check/img/')
            shutil.copy(predict_path, './human_check/label/')
            shutil.copy(predict_path, './human_check/correct_label/')

    else:
        shutil.copy(predict_img, './human_check/img/')
        shutil.copy(predict_path, './human_check/label/')
        shutil.copy(correct_path, './human_check/correct_label/')



In [ ]:
correct_ls = os.listdir('./test_v4/test/labels/')
predict_ls = os.listdir('./runs/detect/predict3/labels')
org_img_path = './test_v4/test/images'

check_txt(correct_ls, predict_ls, org_img_path)

In [51]:
model_verify('./test.txt', './test_v4/test/labels/MP_KSC_000041.txt', 10)

0 0 0 0
0 0 0 0
0 0 0 0


In [56]:
import shutil, os
shutil.rmtree('./human_check/img/')
shutil.rmtree('./human_check/label/')
shutil.rmtree('./human_check/correct_label/')

os.mkdir('./human_check/img')
os.mkdir('./human_check/label')
os.mkdir('./human_check/correct_label')

In [ ]:
import os

predict = './runs/detect/predict3/labels/'
correct = './test_v4/test/labels/'

predict_ls = os.listdir(predict)
correct_ls = os.listdir(correct)

# prdict 된 txt파일만 비교
check = []
for _ in predict_ls:
    if _  in correct_ls:
        check.append(_)

check_predict = []
check_correct = []
for _ in check:
    check_predict.append(predict + _)
    check_correct.append(correct + _)

for one, two in zip(check_predict, check_correct):
    print(one, two)
    # model_verify(one, two, error_rate=10)

In [60]:
one = './human_check/img/'
two = './human_check/label/'
tree = './human_check/correct_label/'

len(os.listdir(one)), len(os.listdir(two)), len(os.listdir(tree))

(1506, 1506, 1506)

In [ ]:
import cv2
img_path = './human_check/img/MP_KSC_000011.jpg'
img = cv2.imread(img_path)

if img is None:
    print('이미지 없습니다.')

cv2.imshow('img', img)
cv2.waitKey(0)

In [5]:
img_path = './human_check/img/MP_KSC_000011.jpg'

show_label('./human_check/correct_label/MP_KSC_000011.txt', img_path)

In [4]:
import cv2, os

def show_label(txt_path, img_path):

    # 원본 txt 파일읽기 - 비율
    with open(txt_path, 'r', encoding='utf-8') as f:
        txt_org = f.readlines()

    # 원본 읽고 불필요 문자 '\n' 제거
    txt_re = []
    for _ in txt_org:
        txt_re.append(_.replace('\n', ''))

    # 이미지 읽기 - 여기서 좌표 변환 후 박스 그리기
    img = cv2.imread(img_path)

    if img is None:
        raise print('이미지 없어 경로 다시확인')

    img_w, img_h, img_c = img.shape

    # 박스 그리는 위치
    for _ in txt_re:
        class_id, x, y, w, h = _.split(' ')
        class_id, x, y, w, h = int(class_id), float(x), float(y), float(w), float(h)

        # yolo식 좌표
        re_x, re_y, re_w, re_h = int(x * img_h), int(y * img_w), int(w * img_h), int(h * img_w)

        # opencv 식 좌표
        x1 = (int(re_x - (re_w/2)), int(re_y + (re_h/2)))
        x2 = (int(re_x + (re_w/2)), int(re_y + (re_h/2)))
        x3 = (int(re_x - (re_w/2)), int(re_y - (re_h/2)))
        x4 = (int(re_x + (re_w/2)), int(re_y - (re_h/2)))

        # print(x1, x4)

        # 클래스 보고 색상변경
        if class_id == 0:
            # 빨 : 사람
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(0, 0, 255), thickness=3)

        elif class_id == 1:
            # 파 : 자전거
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(255, 0, 0), thickness=3)

        elif class_id == 2:
            # 녹 : 차
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(0, 255, 0), thickness=3)

        elif class_id == 3:
            # 핑크 : 오토바이
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(255, 0, 255), thickness=3)

        elif class_id == 4:
            # 주황 : 버스
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(255, 102, 0), thickness=3)

        elif class_id == 5:
            # 노랑 : 트럭
            cv2.rectangle(img=img, pt1=x1, pt2=x4, color=(255, 0, 255), thickness=3)

    # 클래스 개수 적어두기
    cv2.putText(img=img, text=f'total : {len(txt_re)}', org=(0,50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.0, color=(0,0,255), thickness=3)

    cv2.imshow('label', img)

    # key = cv2.waitKey()
    # if key == 27:
    #     cv2.destroyAllWindows()

In [4]:
# txt_path = './human_check/label/MP_KSC_000041.txt'
# txt_path = './test.txt'
# img_path = './human_check/img/MP_KSC_000041.jpg'

img_ls = os.listdir('./human_check/img')
txt_ls = os.listdir('./human_check/correct_label')

for txt, img in zip(txt_ls, img_ls):
    i = './human_check/img/' + img
    t = './human_check/label/' + txt
    show_label(t, i)
    key = cv2.waitKey()
    if key == 27:
        cv2.destroyAllWindows()
        break